Basic random forest classifier using finalised preprocessing pipeline

In [3]:
# different imports
import pandas as pd
import numpy as np
import os
import pickle

#maybe put them with respective code
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification

Loading preprocessed data:

In [4]:
# Import data that has already been preprocessed
data_dir = os.getcwd()
with open(os.path.join(data_dir, "Processing", r"train_preproc.p"), 'rb') as data_file:
    train_data = pickle.load(data_file)
X_train, y_train = train_data[0], train_data[1]

with open(os.path.join(data_dir, "Processing", r"test_preproc.p"), 'rb') as data_file:
    test_data = pickle.load(data_file)
X_test, y_test = test_data[0], test_data[1]


In [5]:
#Training the Algorithm
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(max_depth=7, n_estimators=20, random_state=0)
# parameters to be adjusted to make most optimal 

from ModelEvaluationTools.SKEvaluator import SKEvaluator
rf_model = SKEvaluator(classifier, "rf")


In [6]:
rf_model.fit(X_train, y_train)

In [7]:
test_auc = rf_model.evaluate(X_test, y_test)

Accuracy - Test: 0.9980083333333334
AUC - Test: 0.9291832069024224


Saves the model in the fitted_models folder.

In [8]:
rf_model.save_model()